# OCR on SageMaker--推理

## 1 说明
本章内容为使用SageMaker进行推理，模型来自上一章的训练结果

## 2 运行环境
本文在boto3 1.15.16和sagemaker 2.15.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

如果版本较低，可运行下行代码进行升级，升级完毕后，需要重启Kernal，然后再检查版本

In [ ]:
!pip install boto3 --upgrade -i https://opentuna.cn/pypi/web/simple/
!pip install sagemaker --upgrade -i https://opentuna.cn/pypi/web/simple/

## 3 获取image

本项目已build完毕image，存放到ECR中，可直接部署到SageMaker。请选择选择合适版本。

In [ ]:
#使用CPU版本
image_uri="048912060910.dkr.ecr.cn-northwest-1.amazonaws.com.cn/nwcd/ocr-inference:2.0.2-2.0-4-cpu"

In [ ]:
#使用GPU版本
image_uri="048912060910.dkr.ecr.cn-northwest-1.amazonaws.com.cn/nwcd/ocr-inference:2.0.2-2.0-4-gpu"

## 4 部署到SageMaker
CPU推理目前只能使用large机型，其他机型会报`Intel MKL function load error: cpu specific dynamic library is not loaded.`

In [ ]:
#读取model_data位置
import os
model_data_file = "../1-training/model_data.txt"
if os.path.isfile(model_data_file):
    with open(model_data_file) as f:
        model_data = f.read()
else:
    print("未读取到model_data，请确认已完成训练，将使用默认model")
    model_data="s3://nwcd-samples/nico/model/inference-server.tar.gz"
print(model_data)

In [ ]:
%%time
from sagemaker.model import Model
import boto3

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题
print("ExecutionRole:"+role)

endpoint_name = "ocrtestgpu"
my_model = Model(role=role,image_uri=image_uri,model_data=model_data)

#该步骤,大概需要7-8分钟；根据需要修改instance_type
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.g4dn.2xlarge'
                                )

## 5 推理

### 准备测试图片

In [ ]:
data = {"bucket":"nwcd-samples","image_uri":["nico/data/id.png"]}

返回的结果为textract格式

### 方式一、使用boto3

In [ ]:
import boto3
import json
endpoint_name = "ocrtest"
client = boto3.client('runtime.sagemaker')
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data),
                                  ContentType="application/json")
#print(response)
response_body = response['Body'] 
body= response_body.read()
result = json.loads(body.decode('utf-8'))
print(result)

### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
result = predictor.predict(data)
print(result)

## 6 删除Endpoint

In [ ]:
import boto3
client = boto3.client(service_name='sagemaker') 
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_name)